# 공공데이터포털 실습: 서울 아파트 실거래가

- 지역코드와 거래년월을 입력하면 해당 거래 데이터를 수집하는 사용자 정의 함수를 생성합니다.
- 지역코드와 거래년월을 변경하면 원하는 데이터를 빠르게 수집할 수 있습니다.

### 작업경로 설정

In [ ]:
# 관련 라이브러리를 호출합니다.
import os

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

### 아파트 거래 데이터 수집하는 사용자 정의 함수 생성

In [ ]:
# 공공데이터포털 일반인증키를 설정합니다.
apiKey = '공공데이터포털에서 발급받은 일반인증키를 여기에 붙여넣으세요!'

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import xmltodict
import pandas as pd

# 지역코드와 거래년월을 입력하면 데이터를 수집하는 사용자 정의 함수를 생성합니다.
def aptPrice(areaCd, ymonth, apiKey):
    
    # 요청 URL을 설정합니다.
    url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
    
    # Query String을 생성합니다.
    query = {'LAWD_CD': areaCd, 'DEAL_YMD': ymonth, 'serviceKey': apiKey}
    
    # HTTP 요청을 실행합니다.
    res = requests.get(url = url, params = query)
    
    # str 자료형을 collections.OrderedDict 자료형으로 변환합니다.
    dat = xmltodict.parse(xml_input = res.text)
    
    # dat를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(data = dat['response']['body']['items']['item'])
    
    # 데이터프레임을 반환합니다.
    return df

In [ ]:
# 예제 데이터로 실습
apt = aptPrice(areaCd = '11680', ymonth = '202112', apiKey = apiKey)

In [ ]:
# 처음 다섯 행만 출력합니다.
apt.head()

### 지역코드 준비

In [ ]:
# 서울특별시 자치구 정보가 포함된 엑셀파일을 읽습니다.
dong = pd.read_excel(io = 'Area_Code.xlsx')

In [ ]:
# 처음 10행만 출력합니다.
dong.head(n = 10)

In [ ]:
# 데이터프레임의 정보를 확인합니다.
dong.info()

### 반복문 실행

In [ ]:
# 거래년월을 설정합니다.
ymonth = '202112'

In [ ]:
# 관련 라이브러리를 호출합니다.
# [참고] 웹 크롤링을 실행할 때 잠시 멈춤 동작을 추가해야 합니다.
import time

# 최종 결과를 저장할 빈 데이터프레임을 생성합니다.
apt = pd.DataFrame()

# 데이터프레임 dong의 행 길이를 nrow에 할당합니다.
nrow = len(dong)

# 반복문을 실행합니다.
for i in range(nrow):
    
    # 지역명을 설정합니다.
    areaNm = dong['sido'][i] + ' ' + dong['sigg'][i]
    
    # 현재 진행상황을 출력합니다. 이 코드를 생략하면 진행상황을 알 수 없게 됩니다.
    print(f'[{i+1}/{nrow}] {areaNm} 아파트 매매 데이터 수집 중!')
    
    # 지역코드를 설정합니다. 정수를 문자열로 변경한 다음, 처음 5글자만 잘라냅니다.
    areaCd = dong['code'][i].astype(str)[0:5]
    
    # 거래 건수가 없으면 aptPrice() 함수는 에러를 발생하고 반복문이 중단됩니다.
    # 따라서 try, except 문으로 에러가 발생하면 다음 자치구로 이동시킵니다.
    try:
        
        # 서울특별시 자치구 단위로 거래 데이터를 수집합니다.
        df = aptPrice(areaCd = areaCd, ymonth = ymonth, apiKey = apiKey)
        
        # 필요한 컬럼을 추가합니다.
        df['시도'] = dong['sido'][i]
        df['시군구'] = dong['sigg'][i]
        
        # 최종 결과 데이터프레임에 apt를 추가합니다.
        # [참고] 인덱스가 중복되면 에러가 발생합니다.
        apt = pd.concat(objs = [apt, df], ignore_index = True)
    
    except:
        print('>> 거래 건수가 없습니다. 다음 자치구로 이동합니다!')
        next
    
    # 1초 간 멈춥니다.
    time.sleep(1)

# 데이터 수집 완료
print('>> 모든 데이터를 수집했습니다!')

In [ ]:
# 최종 결과 데이터프레임의 정보를 확인합니다.
apt.info()

In [ ]:
# 시군구별 아파트매매 건수를 확인합니다.
apt.groupby('시군구').count()['거래금액'].sort_values(ascending = False)

### 엑셀 파일로 저장

In [ ]:
# 저장할 엑셀 파일명을 생성합니다.
file = f'APT_Price_Seoul_{ymonth}.xlsx'

In [ ]:
# 엑셀 파일로 저장합니다.
apt.to_excel(excel_writer = file, index = False)

In [ ]:
# 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

## End of Document